In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

%pylab inline

train_df = pd.read_csv('D:/WorkSpace/VisualStudioCode/2022NLP_Course/train_set.csv', sep='\t', nrows=None)
test_df = pd.read_csv('D:/WorkSpace/VisualStudioCode/2022NLP_Course/test_a.csv', sep='\t', nrows=None)

Populating the interactive namespace from numpy and matplotlib


# 数据划分

In [3]:
# hold-out
from sklearn.model_selection import train_test_split

# K折交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

# K折分布保持交叉验证
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

# 时间序列划分方法
from sklearn.model_selection import TimeSeriesSplit

# booststrap 采样
from sklearn.utils import resample

In [4]:
X = np.zeros((20, 5))
Y = np.array([1, 2, 3, 4] * 5)
print(X, Y)

# X = np.zeros((20, 5))
# Y = np.array([1]*5 + [2]*5 + [3]*5 + [4]*5)
# print(X, Y)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] [1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4]


In [5]:
# 直接按照比例拆分
# train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size = 0.2)
# print(train_y, val_y)

# 按照比例 & 标签分布划分
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size = 0.2, stratify=Y)
print(train_y, val_y)

[2 1 4 3 4 1 2 2 4 3 3 1 1 3 4 2] [4 3 2 1]


In [6]:
kf = KFold(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2 3]
Label [1 2 3 4]

[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19] [4 5 6 7]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7 12 13 14 15 16 17 18 19] [ 8  9 10 11]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 16 17 18 19] [12 13 14 15]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] [16 17 18 19]
Label [1 2 3 4]



In [7]:
kf = StratifiedKFold(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2 3]
Label [1 2 3 4]

[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19] [4 5 6 7]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7 12 13 14 15 16 17 18 19] [ 8  9 10 11]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 16 17 18 19] [12 13 14 15]
Label [1 2 3 4]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] [16 17 18 19]
Label [1 2 3 4]



In [8]:
kf = TimeSeriesSplit(n_splits=5)
for train_idx, test_idx, in kf.split(X, Y):
    print(train_idx, test_idx)
    print('Label', Y[test_idx])
    print('')

[0 1 2 3 4] [5 6 7]
Label [2 3 4]

[0 1 2 3 4 5 6 7] [ 8  9 10]
Label [1 2 3]

[ 0  1  2  3  4  5  6  7  8  9 10] [11 12 13]
Label [4 1 2]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] [14 15 16]
Label [3 4 1]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] [17 18 19]
Label [2 3 4]



In [9]:
train_X, train_Y = resample(X, Y, n_samples=16)
val_X, val_Y = resample(X, Y, n_samples=4)
print(train_Y, val_Y)

[2 4 2 1 2 3 4 3 1 1 4 3 4 3 2 3] [3 4 3 4]


# fasttext

In [9]:
train_df = pd.read_csv('D:/WorkSpace/VisualStudioCode/2022NLP_Course/train_set.csv', sep='\t', nrows=None)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')
train_df[['text','label_ft']].iloc[-5000:].to_csv('valid.csv', index=None, header=None, sep='\t')

In [11]:
import fasttext
model = fasttext.train_supervised(input='train.csv',
                                autotuneValidationFile='valid.csv', 
                                autotuneDuration=5)

In [12]:
model.test("valid.csv")

(5000, 0.9236, 0.9236)

# 对抗验证

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_df = pd.read_csv('D:/WorkSpace/VisualStudioCode/2022NLP_Course/train_set.csv', sep='\t', nrows=5000)
test_df = pd.read_csv('D:/WorkSpace/VisualStudioCode/2022NLP_Course/test_a.csv', sep='\t', nrows=5000)

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=500).fit(train_df['text'].iloc[:].values)
train_tfidf = tfidf.transform(train_df['text'].iloc[:].values)
test_tfidf = tfidf.transform(test_df['text'].iloc[:].values)

In [14]:
train_test = np.vstack([train_tfidf.toarray(), test_tfidf.toarray()])

In [18]:
lgb_data = lgb.Dataset(train_test, label=np.array([1]*5000+[0]*5000))

params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['metric'] = 'auc'

result = lgb.cv(params, lgb_data, num_boost_round=100, nfold=3, verbose_eval=20)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5000
[LightGBM] [Info] Number of data points in the train set: 6666, number of used features: 500
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009460 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5000
[LightGBM] [Info] Number of data points in the train set: 6667, number of used features: 500
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5000
[LightGBM] [Info] Number of data points in the train set: 6667, number of used features: 

D:\RuntimeEnvironment\Anaconda\envs\pt\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	cv_agg's auc: 0.491052 + 0.00882533
[40]	cv_agg's auc: 0.491945 + 0.0116601
[60]	cv_agg's auc: 0.493146 + 0.0105694
[80]	cv_agg's auc: 0.494341 + 0.00770488
[100]	cv_agg's auc: 0.495576 + 0.00556407


In [16]:
pd.DataFrame(result)

,auc-mean,auc-stdv
0,0.491807,0.004728
1,0.489903,0.001308
2,0.493536,0.001168
3,0.493294,0.001798
4,0.493753,0.002040
...,...,...
95,0.495292,0.005919
96,0.495490,0.005592
97,0.495188,0.005741
98,0.495539,0.005762
